# Study cohort selection and variable extraction

In [1]:
# Imports:
import psycopg2
import numpy as np
import pandas as pd
import os
import yaml

In [2]:
# Connect to the mimic database:
sqluser = 'postgres'
password='postgres'
dbname = 'mimic'
schema_name = 'public, mimic, mimiciii;'

con = psycopg2.connect(dbname=dbname, user=sqluser, password=password)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)
# cur.close()
# con.close()

### Study cohort selection
- Only first ICU admissions that took at least a day and less than 10 days
- Adult patients only (age >= 15)

In [3]:
# Settings for the query:
min_age = 15
limit_population = 0 # if we want to run the query for a small number of patients (for debugging)
if limit_population > 0:
    limit = 'LIMIT ' + str(limit_population)
else:
    limit = ''

In [4]:
query = """
with patient_and_icustay_details as (
    SELECT distinct
        p.gender, p.dob, p.dod, s.*, a.admittime, a.dischtime, a.deathtime, a.ethnicity, a.diagnosis,
        DENSE_RANK() OVER (PARTITION BY a.subject_id ORDER BY a.admittime) AS hospstay_seq,
        DENSE_RANK() OVER (PARTITION BY s.hadm_id ORDER BY s.intime) AS icustay_seq,
        DATE_PART('year', s.intime) - DATE_PART('year', p.dob) as admission_age,
        DATE_PART('day', s.outtime - s.intime) as los_icu
    FROM patients p 
        INNER JOIN icustays s ON p.subject_id = s.subject_id
        INNER JOIN admissions a ON s.hadm_id = a.hadm_id 
    WHERE s.first_careunit NOT like 'NICU'
        and s.hadm_id is not null and s.icustay_id is not null
        and (s.outtime >= (s.intime + interval '12 hours'))
        and (s.outtime <= (s.intime + interval '240 hours'))
    ORDER BY s.subject_id 
)
SELECT * 
FROM patient_and_icustay_details 
WHERE hospstay_seq = 1
    and icustay_seq = 1
    and admission_age >=  """ + str(min_age) + """
    and los_icu >= 0.5
""" + str(limit)
patients_data = pd.read_sql_query('SET search_path to ' + schema_name + query, con)

# Save result:
#patients_data.to_csv('static_data.csv')

In [5]:
patients_data

,gender,dob,dod,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,...,los,admittime,dischtime,deathtime,ethnicity,diagnosis,hospstay_seq,icustay_seq,admission_age,los_icu
0,M,2025-04-11,2102-06-14,2,3,145834,211552,carevue,MICU,MICU,...,6.0646,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,WHITE,HYPOTENSION,1,1,76.0,6.0
1,F,2143-05-12,NaT,3,4,185777,294638,carevue,MICU,MICU,...,1.6785,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",1,1,48.0,1.0
2,F,2109-06-21,NaT,5,6,107064,228232,carevue,SICU,SICU,...,3.6729,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,WHITE,CHRONIC RENAL FAILURE/SDA,1,1,66.0,3.0
3,M,2108-01-26,2149-11-14,9,9,150750,220597,carevue,MICU,MICU,...,5.3231,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,UNKNOWN/NOT SPECIFIED,HEMORRHAGIC CVA,1,1,41.0,5.0
4,F,2128-02-22,2178-11-14,11,11,194540,229441,carevue,SICU,SICU,...,1.5844,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,WHITE,BRAIN MASS,1,1,50.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30058,M,2114-09-29,NaT,61527,99983,117390,286606,metavision,CCU,CCU,...,1.0399,2193-04-26 11:35:00,2193-04-29 13:30:00,NaT,UNKNOWN/NOT SPECIFIED,ST ELEVATION MYOCARDIAL INFARCTION;CORONARY AR...,1,1,79.0,1.0
30059,M,2137-04-07,NaT,61529,99991,151118,226241,metavision,TSICU,TSICU,...,3.1426,2184-12-24 08:30:00,2185-01-05 12:15:00,NaT,WHITE,DIVERTICULITIS/SDA,1,1,47.0,3.0
30060,F,2078-10-17,NaT,61530,99992,197084,242052,metavision,MICU,MICU,...,1.9745,2144-07-25 18:03:00,2144-07-28 17:56:00,NaT,WHITE,RETROPERITONEAL HEMORRHAGE,1,1,66.0,1.0
30061,F,2058-05-29,2147-09-29,61531,99995,137810,229633,metavision,CSRU,CSRU,...,2.1615,2147-02-08 08:00:00,2147-02-11 13:15:00,NaT,WHITE,ABDOMINAL AORTIC ANEURYSM/SDA,1,1,89.0,2.0


### Extraction of vital data and mapping to variables

In [9]:
variables_to_keep = ('Capillary refill rate', 'Diastolic blood pressure', 'Fraction inspired oxygen', 
                     'Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total',
                     'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure',
                     'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH')
var_map = pd.read_csv('../resources/itemid_to_variable_map.csv')

In [10]:
icu_ids_to_keep = patients_data['icustay_id']
icu_ids_to_keep = tuple(set([str(i) for i in icu_ids_to_keep]))
subjects_to_keep = patients_data['subject_id']
subjects_to_keep = tuple(set([str(i) for i in subjects_to_keep]))
hadms_to_keep = patients_data['hadm_id']
hadms_to_keep = tuple(set([str(i) for i in hadms_to_keep]))

labitems_to_keep = []
chartitems_to_keep = []
for i in range(var_map.shape[0]):
    if var_map['LEVEL2'][i] in variables_to_keep:
        if var_map['LINKSTO'][i] == 'chartevents':
            chartitems_to_keep.append(var_map['ITEMID'][i])
        elif var_map['LINKSTO'][i] == 'labevents':
            labitems_to_keep.append(var_map['ITEMID'][i])
            
all_to_keep = chartitems_to_keep + labitems_to_keep
var_map = var_map[var_map.ITEMID.isin(all_to_keep)]
chartitems_to_keep = tuple(set([str(i) for i in chartitems_to_keep]))
labitems_to_keep = tuple(set([str(i) for i in labitems_to_keep]))

In [ ]:
query = """
SELECT c.subject_id, i.hadm_id, c.icustay_id, c.charttime, c.itemid, c.value, c.valueuom
FROM icustays i
INNER JOIN chartevents c ON i.icustay_id = c.icustay_id
where c.icustay_id in """ + str(icu_ids_to_keep) + """
  and c.itemid in """ + str(chartitems_to_keep) + """
  and c.charttime between intime and outtime
  and c.error is distinct from 1
  and c.valuenum is not null
UNION ALL
SELECT distinct i.subject_id, i.hadm_id, i.icustay_id, l.charttime, l.itemid, l.value, l.valueuom
FROM icustays i
INNER JOIN labevents l ON i.hadm_id = l.hadm_id
where i.icustay_id in """ + str(icu_ids_to_keep) + """
  and l.itemid in """ + str(labitems_to_keep) + """
  and l.charttime between (intime - interval '6' hour) and outtime
  and l.valuenum > 0 -- lab values cannot be 0 and cannot be negative
"""
events_data = pd.read_sql_query('SET search_path to ' + schema_name + query, con)

In [11]:
itemids = tuple(set(events_data.itemid.astype(str)))
events_data.head()

,subject_id,hadm_id,icustay_id,charttime,itemid,value,valueuom
0,4588,186721,200746,2113-03-04 16:00:00,224167,100,mmHg
1,4588,186721,200746,2113-03-04 19:00:00,224167,90,mmHg
2,6214,169579,224823,2131-02-22 20:00:00,224167,90,mmHg
3,6214,169579,224823,2131-02-26 10:57:00,224167,88,mmHg
4,6214,169579,224823,2131-02-22 10:55:00,224167,98,mmHg


In [12]:
query_d_items = \
        """
        SELECT itemid, label, dbsource, linksto, category, unitname
        FROM d_items
        WHERE itemid in """ + str(itemids)
d_output = pd.read_sql_query('SET search_path to ' + schema_name + query_d_items, con)

In [24]:
# Remove the text from the categorical (Glasgow coma scale) variables so we can make them numeric:
replacement_dictionary = {'4 Spontaneously': '4', '3 To speech': '3', '2 To pain': '2', '1 No Response': '1',
                         '5 Oriented': '5', '1.0 ET/Trach': '1', '4 Confused': '4', '2 Incomp sounds': '2', 
                         '3 Inapprop words': '3', 'Spontaneously': '4', 'To Speech': '3', 'None': '1', 'To Pain': '2',
                         '6 Obeys Commands': '6', '5 Localizes Pain': '5', '4 Flex-withdraws': '4', '2 Abnorm extensn': '2',
                         '3 Abnorm flexion': '3', 'No Response-ETT': '1', 'Oriented': '5', 'Confused': '4', 
                         'No Response': '1', 'Incomprehensible sounds': '2', 'Inappropriate Words': '3', 
                         'Obeys Commands': '6', 'No response': '1', 'Localizes Pain': '5', 'Flex-withdraws': '4',
                         'Abnormal extension': '2', 'Abnormal flexion': '3', 'Abnormal Flexion': '3', 
                          'Abnormal Extension': '2'}
for key, value in replacement_dictionary.items():
    events_data['value'] = events_data['value'].replace(key, value) 

In [15]:
# Change data types and set indices:
events_data['value'] = pd.to_numeric(events_data['value']) #, 'coerce')
events_data = events_data.astype({k: int for k in ['subject_id', 'hadm_id', 'icustay_id']})
patients_data = patients_data.reset_index().set_index('icustay_id')
var_map = var_map[['LEVEL2', 'ITEMID', 'LEVEL1']].rename(
    {'LEVEL2': 'LEVEL2', 'LEVEL1': 'LEVEL1', 'ITEMID': 'itemid'}, axis=1).set_index('itemid')

# Change to hourly data:
to_hours = lambda x: max(0, x.days*24 + x.seconds // 3600)
events_data = events_data.set_index('icustay_id').join(patients_data[['intime']])
events_data['hours_in'] = (events_data['charttime'] - events_data['intime']).apply(to_hours)
events_data = events_data.drop(columns=['charttime', 'intime']) 

# Join with d_output query and group variables:
events_data = events_data.set_index('itemid', append=True)
events_data = events_data.join(var_map)
d_output = d_output.set_index('itemid')
events_data = events_data.join(d_output) 
events_data = events_data.set_index(['label', 'LEVEL1', 'LEVEL2'], append=True)
patients_data['max_hours'] = (patients_data['outtime'] - patients_data['intime']).apply(to_hours)

In [30]:
# Save results:
np.save('subjects.npy', patients_data['subject_id'])
np.save('times_hours.npy', patients_data['max_hours'])
patients_data.to_hdf('vitals_hourly_data.h5', 'patients_data')
events_data.to_hdf('vitals_hourly_data.h5', 'X')

C:\Users\hester\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:2605: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['gender', 'dbsource', 'first_careunit', 'last_careunit', 'ethnicity',
       'diagnosis'],
      dtype='object')]

  pytables.to_hdf(
C:\Users\hester\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:2605: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['valueuom', 'dbsource', 'linksto', 'category', 'unitname'], dtype='object')]

  pytables.to_hdf(


### Extract length of stay and in-hospital mortality

In [ ]:
outcomes = pd.DataFrame(index=patients_data.index)
# In hospital mortality: patient has died after the admittime to hospital and before the outtime:
mortality = patients_data.dod.notnull() & ((patients_data.admittime <= patients_data.dod) & (patients_data.outtime >= patients_data.dod))
mortality = mortality | (patients_data.deathtime.notnull() & ((patients_data.admittime <= patients_data.deathtime) & 
                                                             (patients_data.dischtime >= patients_data.deathtime)))
outcomes['in_hospital_mortality'] = mortality.astype(int)

# Length of stay (in hours):
outcomes['los'] = patients_data['los'] * 24.0
outcomes.to_hdf('vitals_hourly_data.h5', 'Y')